# О задании

## Тема
Объединение таблиц и сложные выборки

## Цель домашнего задания
- научиться работать с интервальными таблицы, уметь их джойнить с другими таблицами;
- научиться обрабатывать транзакции относительно какого-то промежутка времени.
 

## Условия
Дано 2 CSV-файла: один с транзакциями за период 01.01.2023 – 20.04.2023, второй с информацией о клиентах.

**За дефолтную дату для выборки действующих клиентов брать ‘2023-05-01’**

Необходимо в Jupyter- ноутбуке выполнить следующие пункты, используя SQLite:

<font size=4>[**Шаг 1**](#Шаг-1)</font>

(Балл - 2) Необходимо скачать CSV-файл - «[transactions_for_dz2.csv](https://drive.google.com/file/d/1GsKxPQ3cp1_uuyoLoAQLlsHTLEbXOMIo/view?usp=share_link)», создать таблицу *transaction_bd* со всеми полями, загрузить данные из файла в таблицу и оставить таблицу со структурой:

(0, 'TRANSACTION_ID', 'INTEGER', 0, None, 0) - id транзакции

(1, 'TX_DATETIME', 'NUMERIC', 0, None, 0) - дата транзакции

(2, 'CUSTOMER_ID', 'INTEGER', 0, None, 0) - id клиента

(3, 'TERMINAL_ID', 'INTEGER', 0, None, 0) - id терминала

(4, 'TX_AMOUNT', 'REAL', 0, None, 0) - сумма транзакции

Также необходимо скачать второй CSV-файл - «[client_info.csv](https://drive.google.com/file/d/1oFzZfwHBtP0NOulBYlp4osIY5NIPIdn-/view?usp=share_link)», создать таблицу *customer_bd* со всеми полями, загрузить данные из файла в таблицу и получить таблицу со структурой:

(0, 'CLIENT_ID', 'INTEGER', 0, None, 0) - id клиента

(1, 'START_DT', 'NUMERIC', 0, None, 0) - дата начало записи о клиенте

(2, 'END_DT', 'NUMERIC', 0, None, 0) - дата закрытия записи о клиенте

(3, 'CLIENT_NAME', 'TEXT', 0, None, 0) - название клиента

(4, 'YEAR_BIRTH', 'TEXT', 0, None, 0) - дата рождение клиента

**Примечание для следующих шагов:** сумма транзакций не может быть null-ом, это всегда число.

<font size=4>[**Шаг 2**](#Шаг-2)</font>

**(Балл - 1 за каждый пункт)** Написать скрипты:

- a. Найти имя клиента/клиентов с максимальной суммой транзакций за весь период (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).
- b. Найти имя клиента/клиентов с минимальной суммой транзакций за весь период (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).
- c. Найти сумму транзакций относительно даты 2023-04-01 для клиентов, у которых id начинается с 111 (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся). Транзакции учитываются только после завершении дня.
- d. Найти сумму транзакций относительно года рождения клиентов (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся). Сортировать по убыванию года рождения.
- e. Найти количество транзакций для каждого клиента (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся). Сортировать по убыванию количества транзакций.

<font size=4>[**Шаг 3**](#Шаг-3)</font>

**(Балл - 3)** Найти сумму транзакций за каждый месяц (январь, февраль, март, апрель) для всех клиентов (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).
 

## Сроки выполнения задания
До 09.04.2023, 23:59.

In [62]:
import pandas as pd
import sqlite3
from datetime import datetime
import re

## Шаг 1

In [20]:
transaction_bd = pd.read_csv('transactions_for_dz2.csv', parse_dates=['TX_DATETIME'])

In [21]:
transaction_bd.head(3)

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT
0,0,2023-01-01 00:00:31,596,3156,533.07
1,1,2023-01-01 00:02:10,4961,3412,808.56
2,2,2023-01-01 00:07:56,2,1365,1442.94


In [22]:
transaction_bd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 5 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   TRANSACTION_ID  1048575 non-null  int64         
 1   TX_DATETIME     1048575 non-null  datetime64[ns]
 2   CUSTOMER_ID     1048575 non-null  int64         
 3   TERMINAL_ID     1048575 non-null  int64         
 4   TX_AMOUNT       1048575 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 40.0 MB


In [59]:
customer_bd = pd.read_csv('client_info.csv', sep=';', nrows=2, parse_dates=[0, 1])#, keep_default_na=True)

In [60]:
customer_bd.head(3)

,START_DT,END_DT,CLIENT_NAME,YEAR_BIRTH,CLIENT_ID
0,2015-07-16,2021-01-01,Olivia,1979,2213
1,2015-07-17,2021-01-01,Emma,1979,1148


In [61]:
customer_bd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   START_DT     2 non-null      datetime64[ns]
 1   END_DT       2 non-null      datetime64[ns]
 2   CLIENT_NAME  2 non-null      object        
 3   YEAR_BIRTH   2 non-null      int64         
 4   CLIENT_ID    2 non-null      int64         
dtypes: datetime64[ns](2), int64(2), object(1)
memory usage: 208.0+ bytes


In [ ]:
customer_bd['START_DT'].dt.

In [63]:
pattern = '\d\d\d\d-\d\d-\d\d'

In [75]:
customer_bd.loc[:, 'END_DT']#.apply(lambda x: re.match(pattern, x))

0   2021-01-01
1   2021-01-01
Name: END_DT, dtype: datetime64[ns]

In [67]:
help(re.match)

Help on function match in module re:

match(pattern, string, flags=0)
    Try to apply the pattern at the start of the string, returning
    a Match object, or None if no match was found.



In [49]:
pd.read_csv?

Создадим функции для подключения к базе данных, выполнения запросов и создании таблиц.

In [9]:
def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [10]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

In [11]:
def sql_insert(con, entities, text_sql_insert_into_table):

    cursorObj = con.cursor()
    cursorObj.execute(text_sql_insert_into_table, entities)

Подключимся к базе данных

In [12]:
connection = create_connection('database.db')

Connection to SQLite DB successful


Создадим таблицу transaction_bd

In [127]:
sql_create_table = '''
                        CREATE TABLE IF NOT EXISTS transaction_bd (
                                            "TRANSACTION_ID" INTEGER,
                                            "TX_DATETIME" NUMERIC,
                                            "CUSTOMER_ID" INTEGER,
                                            "TERMINAL_ID" INTEGER,
                                            "TX_AMOUNT" REAL
                                                        )
                     '''

In [128]:
execute_query(connection, sql_create_table)

Query executed successfully


Создадим таблицу customer_bd

In [129]:
sql_create_table = '''
                        CREATE TABLE IF NOT EXISTS customer_bd (
                                            "CLIENT_ID" INTEGER,
                                            "START_DT" NUMERIC,
                                            "END_DT" NUMERIC,
                                            "CLIENT_NAME" TEXT,
                                            "YEAR_BIRTH" TEXT
                                                        )
                     '''

In [130]:
execute_query(connection, sql_create_table)

Query executed successfully


Прочитаем данные из файлов и вставим их в таблицы

In [131]:
text_sql_insert_into_table = '''
                                 INSERT INTO transaction_bd(
                                                         TRANSACTION_ID,
                                                         TX_DATETIME,
                                                         CUSTOMER_ID,
                                                         TERMINAL_ID,
                                                         TX_AMOUNT
                                                      )
                                 VALUES(?, ?, ?, ?, ?)
                             '''

In [132]:
with open('transactions_for_dz2.csv', encoding='utf-8') as file:
    text = file.readlines()
for line in text[1:]:
    sql_insert(connection, line.split(','), text_sql_insert_into_table)
connection.commit()

In [133]:
text_sql_insert_into_table = '''
                                 INSERT INTO customer_bd(
                                                         START_DT,
                                                         END_DT,
                                                         CLIENT_NAME,
                                                         YEAR_BIRTH,
                                                         CLIENT_ID
                                                      )
                                 VALUES(?, ?, ?, ?, ?)
                             '''

In [134]:
with open('client_info.csv', encoding='utf-8') as file:
    text = file.readlines()
for line in text[1:]:
    sql_insert(connection, line.split(';'), text_sql_insert_into_table)
connection.commit()

## Шаг 2
Напишем скрипты:

a. Найти имя клиента/клиентов с максимальной суммой транзакций за весь период (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).

In [13]:
transaction_bd.head(3)

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT
0,0,2023-01-01 00:00:31,596,3156,533.07
1,1,2023-01-01 00:02:10,4961,3412,808.56
2,2,2023-01-01 00:07:56,2,1365,1442.94


In [14]:
customer_bd.head()

,START_DT,END_DT,CLIENT_NAME,YEAR_BIRTH,CLIENT_ID
0,2015-07-16,2021-01-01,Olivia,1979,2213
1,2015-07-17,2021-01-01,Emma,1979,1148
2,2015-07-18,2021-01-01,Charlotte,1979,2293
3,2015-07-19,2021-01-01,Amelia,1979,1867
4,2015-07-20,2021-01-01,Ava,1979,1767


In [16]:
df_merge = pd.merge(transaction_bd, customer_bd, left_on='CUSTOMER_ID', right_on='CLIENT_ID')

In [18]:
df_merge.head(3)

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,START_DT,END_DT,CLIENT_NAME,YEAR_BIRTH,CLIENT_ID
0,0,2023-01-01 00:00:31,596,3156,533.07,2011-03-03,2023-03-03,Janice,1957,596
1,6440,2023-01-01 14:32:25,596,7633,558.99,2011-03-03,2023-03-03,Janice,1957,596
2,8491,2023-01-01 18:33:13,596,2440,589.15,2011-03-03,2023-03-03,Janice,1957,596


In [19]:
df_merge[df_merge['END_DT'] >= datetime(2023, 5, 1)]

TypeError: '>=' not supported between instances of 'str' and 'datetime.datetime'

b. Найти имя клиента/клиентов с минимальной суммой транзакций за весь период (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).
c. Найти сумму транзакций относительно даты 2023-04-01 для клиентов, у которых id начинается с 111 (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся). Транзакции учитываются только после завершении дня.
d. Найти сумму транзакций относительно года рождения клиентов (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся). Сортировать по убыванию года рождения.
e. Найти количество транзакций для каждого клиента (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся). Сортировать по убыванию количества транзакций.

## Шаг 3

In [135]:
connection.close()